In [ ]:
from sentence_transformers import SentenceTransformer

bert = SentenceTransformer('all-MiniLM-L6-v2')

bert_vectors = bert.encode(df['combined'].tolist(), convert_to_numpy=True)

X_bert = np.concatenate([X_scaled, bert_vectors], axis=1)

knn_bert = NearestNeighbors(n_neighbors=5, metric='cosine').fit(X_bert)

def recommend_items_bert(user_input, nutritions, df, scaler, bert_model, knn_model):
    # 1. Generate BERT embedding for input text
    text_vec = bert.encode([user_input], convert_to_numpy=True)  # shape: (1, D)
    df_test=pd.DataFrame.from_dict(nutritions)
    # 2. Scale nutrition input to match training distribution
    nutrition_vec = scaler.transform(df_test.values)  # shape: (1, 4)

    # 3. Concatenate scaled nutrition + text embeddings
    query_vec = np.concatenate([nutrition_vec, text_vec], axis=1)  # shape: (1, D+4)

    # 4. Retrieve top_k nearest neighbors
    distances, indices = knn_model.kneighbors(query_vec, n_neighbors=5)

    # 5. Create results DataFrame
    recommendations = [
        (df1.iloc[idx], distances[0][i])
        for i, idx in enumerate(indices[0])
    ]
    return recommendations


user_input = "oats banana, salad , sweet"
  # calories, protein, fat, carbs
nutritions_goals={"calories":[100],"proteins":[40.0],"carbohydrates":[45.00],"fiber":[15.00],"fat":[16.00],"sodium":[8.00]}
recommendations = recommend_items_bert(user_input,nutritions_goals,df1,scaler,bert,knn_bert)
# print(recommendations)
for i, (name, distance) in enumerate(recommendations):
    print(f"{i+1}. {name}")